<a href="https://colab.research.google.com/github/brunorosilva/ai_especialization_usp/blob/master/IAD-004-Aprendizagem_de_Maquina_1/Exercicios_cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import decomposition
from sklearn import cluster
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn import metrics
import matplotlib as mpl
import matplotlib.pyplot as plt
import urllib.request as request
import io
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
%matplotlib inline 

# Organizando um evento científico com aprendizagem não-supervisionada.

A aprendizagem não-supervisionada é capaz de encontrar redundâncias e similaridades em um conjunto de dados.

Vamos usar esta capacidade para organizar sessões do simpósio de trabalhos de conclusão de curso de Engenharia Mecatrônica de 2019.

O código abaixo recupera a planilha construída pelo Prof. Thiago Martins com a lista de todos os trabalhos e as palavras-chave associadas a cada um.

In [ ]:
!wget -O trabalhos_2019.xls https://www.lsc.poli.usp.br/trabalhos_2019.xls

--2020-07-22 00:10:06--  https://www.lsc.poli.usp.br/trabalhos_2019.xls
Resolving www.lsc.poli.usp.br (www.lsc.poli.usp.br)... 143.107.99.33
Connecting to www.lsc.poli.usp.br (www.lsc.poli.usp.br)|143.107.99.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18944 (18K) [application/vnd.ms-excel]
Saving to: ‘trabalhos_2019.xls’

trabalhos_2019.xls  100%[===================>]  18.50K  63.8KB/s    in 0.3s    

2020-07-22 00:10:09 (63.8 KB/s) - ‘trabalhos_2019.xls’ saved [18944/18944]



A planilha tem 3 sub-planilhas, a planilha ```trabalho``` com a lista de cada trabalho, a planilha ```palavras-chave``` com a lista de todas as palavras-chave e a planilha ```p_t``` que contém todas as associações de palavra-chave a trabalho.

In [ ]:
planilha=pd.ExcelFile('trabalhos_2019.xls')
trabalhos=pd.read_excel(planilha,'trabalhos')
palavras_chave=pd.read_excel(planilha, 'palavras-chave')
palavras_trabalhos=pd.read_excel(planilha, 'p_t')

Eis alguns trabalhos:

In [ ]:
trabalhos.head()

,Grupo,Título
0,1,Estudo de soluções para o caso de impacto veic...
1,2,Projeto de Estabilizador de Câmera via Platafo...
2,3,Controlador para Exoesqueleto Unilateral Modul...
3,4,Proposta de virtualização do processo de comis...
4,5,Desenvolvimento de um sistema inteligente de C...


Eis algumas palavras-chave:

In [ ]:
palavras_chave.head()

,#,Palavra-chave
0,1,FEM
1,2,veículos automotivos
2,3,segurança física
3,4,anatomia
4,5,controle


A base ```palavras_trabalhos``` faz a associação entre estas bases.

In [ ]:
palavras_trabalhos.head()

,grupo,palavra
0,1,1
1,6,1
2,11,1
3,12,1
4,13,1


Por exemplo, eis todas as palavras-chave associadas com o trabalho do grupo 1:

In [ ]:
palavras_trabalhos[palavras_trabalhos['grupo']==1].merge(palavras_chave, left_on='palavra', right_on='#')[['#','Palavra-chave']]

,#,Palavra-chave
0,1,FEM
1,2,veículos automotivos
2,3,segurança física
3,4,anatomia
4,7,engenharia biomédica
5,17,Projeto Mecânico
6,24,Transporte e mobilidade


Vamos atribuir a cada grupo um vetor de características.
Este vetor tem uma componente para cada palavra-chave.
Ele vai ter 1 na componente $i$ se a $i$-gésima palavra-chave está associada ao trabalho, ou 0 caso contrário.

O código abaixo monta este vetor para um grupo:

In [ ]:
def vetor_grupo(grupo):
  palavras = palavras_trabalhos[palavras_trabalhos['grupo']==grupo]['palavra']
  vetor = np.zeros(palavras_chave.shape[0])
  for i in palavras:
    vetor[i-1] = 1
  return vetor

Exemplo: O vetor característico do grupo 1.

In [ ]:
vetor_grupo(1)

array([1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

## Construindo um agrupamento de trabalhos

### 1. Redução de dimensão

Em primeiro lugar vamos reduzir a dimensão do problema.
Como pode-se ver pela lista de palavras, várias são fortemente correlacionadas (e eventualmente redundantes), como "veículos automotivos" e "Transporte e mobilidade".

Para mitigar isso, vamos fazer uma decomposição em componentes principais reduzindo a dimensão do problema para 10.

Gere vetores característicos para cada grupo, faça uma decomposição em componentes principais retendo os 10 primeiros vetores e projete os vetores característicos neste espaço reduzido.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
dados = []
for i in range(1, 35):
  dados.append(vetor_grupo(i).tolist())

pca = PCA(n_components=10)
pca.fit(dados)

#pca.explained_variance_

dados_projetados = pca.transform(dados)

### 2. Agrupamento.

Use o objeto KMeans da biblioteca SKLearn para agrupar os vetores do espaço reduzido em 8 clusters.

Descubra quais os números dos trabalhos correspondentes ao cluster com a *menor* quantidade de trabalhos.

In [ ]:
modelo_kmeans = cluster.KMeans(n_clusters=8, n_init=60)
modelo_kmeans.fit(dados_projetados)

df_labels = pd.DataFrame(modelo_kmeans.labels_)

df_labels[0].value_counts()


3    6
7    5
2    5
1    5
0    5
5    3
4    3
6    2
Name: 0, dtype: int64

In [ ]:
df_labels[df_labels[0] == 3]

,0
1,3
7,3
9,3
14,3
20,3
28,3


In [ ]:
for i in (1, 7, 9, 14, 20, 28):
  print(trabalhos.iloc[i]["Título"])

Projeto de Estabilizador de Câmera via Plataforma de Stewart para Veículos Autônomos ou Pilotados Remotamente
CONTROLE DE ROBÔ MANIPULADOR POR MEIO DE MÉTODOS DE INTELIGÊNCIA ARTIFICIAL
Módulo de potência para motores turbo
Simulação de Direção de Veículo Autônomo Utilizando Redes Neurais Convolucionais e Recorrentes
Proteção cambial de operações financeiras baseada em aprendizado de máquina
A Machine Learning Approach to Cycling User Classification


In [ ]:
trabalhos.iloc[2]["Título"]

'Controlador para Exoesqueleto Unilateral Modular com 1 e 2 Graus de Liberdade '

In [ ]:
trabalhos.iloc[30]["Título"]

'PROJETO E IMPLEMENTAÇÃO DE CONTROLADOR DE POSIÇÃO PARA PROTÓTIPO DE SIMULADOR DE MEMBRO SUPERIOR HUMANO'